In [1]:
from src import environments
import nest_asyncio
nest_asyncio.apply()

from gym_derk.envs import DerkEnv
import gym_derk
from tensorflow.keras import layers
from tensorflow import keras
import numpy as np
from src.environments import EnvTensorOne
import tf_agents

2023-07-01 09:22:36.656514: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
env = EnvTensorOne()

In [41]:
env.close()

In [4]:
tf_env = tf_agents.environments.tf_py_environment.TFPyEnvironment(env)

In [46]:
env.reset()
action = np.random.rand(15)
a = env.step(action)

In [ ]:
a

In [4]:
from tf_agents.environments import utils
from tf_agents.environments import tf_py_environment

In [5]:
utils.validate_py_environment(env, episodes=2)

ValueError: Given `time_step`: TimeStep(
{'discount': array(1., dtype=float32),
 'observation': array([ 1.        ,  0.        ,  0.36897796,  0.56313133,  0.09551312,
       -0.45030487,  0.07049811, -0.6402546 ,  0.86756545, -0.52053714,
        0.25701886, -0.57274437,  2.        ,  0.        ,  0.15074064,
       -0.44643545,  1.        , -0.4636289 , -0.99854386,  0.        ,
        0.22645962,  1.        ,  1.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        , -0.02238032, -0.08336428,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
        0.        ,  1.        ,  1.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
        0.        ,  0.        ,  0.    